In [91]:
import pandas as pd 
import numpy as np

In [92]:
df = pd.read_csv('train (1).csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [94]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [95]:
df.sample(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
285,0,3,male,33.0,0,0,8.6625,C
474,0,3,female,22.0,0,0,9.8375,S
493,0,1,male,71.0,0,0,49.5042,C


In [96]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [97]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [99]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [100]:
print('X_train',X_train.shape)
print('X_test',X_test.shape)

X_train (712, 7)
X_test (179, 7)


In [101]:
from sklearn.compose import ColumnTransformer

In [102]:
# imputation transformer
from sklearn.impute import SimpleImputer

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(strategy='mean'),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6]),
],remainder='passthrough')

In [103]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]), 
],remainder='passthrough')

In [104]:
# Scaling
from sklearn.preprocessing import MinMaxScaler

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [105]:
# Feature selection
from sklearn.feature_selection import SelectKBest,chi2

trf4 = SelectKBest(score_func=chi2,k=8)

In [106]:
# train the model
from sklearn.tree import DecisionTreeClassifier

trf5 = DecisionTreeClassifier()


Create Pipeline

In [128]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

Pipeline Vs make_pipeline
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [108]:
# Alternate Syntax
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [109]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001D922BEB400>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

Explore the Pipeline

In [110]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000001D922BEB400>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [111]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [112]:
# Predict

y_pred = pipe.predict(X_test)

y_pred.shape

(179,)

In [113]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.6256983240223464

Cross Validation using Pipeline

In [114]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score

cross_val_score(pipe,X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

GridSearch using Pipeline

In [123]:
('trf5', DecisionTreeClassifier())

('trf5', DecisionTreeClassifier())

In [ ]:
from sklearn.model_selection import GridSearchCV

# gridsearchcv
params = {
    'trf5__max_depth': [1, 2, 3, 4, 5, None]
}
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001D922BEB400>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

Exporting the Pipeline


In [131]:
# export 

import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))